# San Francisco Housing Cost Analysis

In this assignment, you will perform fundamental analysis for the San Francisco housing market to allow potential real estate investors to choose rental investment properties.

In [1]:
# Initial imports of libraries and their dependencies
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv


import warnings
warnings.filterwarnings('ignore')

AttributeError: module 'pandas' has no attribute 'Panel'

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")

# Print type of map_box_api to check if the class is 'str'
print(type(map_box_api))

## Load Data

In [ ]:
# Read the census data into a Pandas DataFrame
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()

- - - 

## Housing Units Per Year

In this section, you will calculate the number of housing units per year and visualize the results as a bar chart using the Pandas plot function.

**Hint:** Use the Pandas `groupby` function.

**Optional challenge:** Use the min, max, and std to scale the y limits of the chart.



In [ ]:
# Calculate the mean number of housing units per year (hint: use groupby) 

housing_units_grp = (
    sfo_data['housing_units'].groupby(['year']).mean()
)
housing_units_grp

In [ ]:
# Save the dataframe as a csv file
sfo_housing_units_df = pd.DataFrame(housing_units_grp).reset_index()
sfo_housing_units_df

## Optional Challenge: Use the min, max, and std to scale the y limits of the chart

In [ ]:
# Leveraged MIN, MAX, and STD functions to scale the Y limits of the chart

housing_units_std = sfo_housing_units_df.housing_units.std()
housing_units_min_1 = min(sfo_housing_units_df.housing_units)
housing_units_max_1 = max(sfo_housing_units_df.housing_units)

sfo_housing_units_df.hvplot.bar(x = 'year', y = 'housing_units', 
                                ylim=(housing_units_min_1 - housing_units_std, housing_units_max_1 + housing_units_std), 
                                title = "Housing Units in San Francisco from 2010 to 2016",
                                xlabel = 'Year', ylabel = 'Housing Units', yformatter="%.0f"
                               )

- - - 

## Average Housing Costs in San Francisco Per Year

In this section, you will calculate the average monthly rent and the average price per square foot for each year. An investor may wish to better understand the sales price of the rental property over time. For example, a customer will want to know if they should expect an increase or decrease in the property value over time so they can determine how long to hold the rental property.  Plot the results as two line charts.

**Optional challenge:** Plot each line chart in a different color.

In [ ]:
# Calculate the average sale price per square foot and average gross rent
# 'year' is set as index, and groupby mean function will calculate mean values per year for columns
# 'sale_price_sqr_foot' and 'gross_rent'

average_price_group = (
    sfo_data[['sale_price_sqr_foot', 'gross_rent']].groupby(['year']).mean()
)
average_price_group

In [ ]:
# Create two line charts, one to plot the average sale price per square foot 
# and another for average monthly rent.
# Calculations and line chart for 'average sale price per square foot'

sqr_foot_df = average_price_group['sale_price_sqr_foot'].reset_index()

sqr_std = sqr_foot_df.sale_price_sqr_foot.std()
sqr_min = min(sqr_foot_df.sale_price_sqr_foot) - sqr_std
sqr_max = max(sqr_foot_df.sale_price_sqr_foot) + sqr_std

# Line chart for average sale price per square foot + optional challenge to change the line color

sqr_foot_df.hvplot.line(x='year', y='sale_price_sqr_foot',title= "Average Sale Price Per SqFt by Year", 
                        ylim = (sqr_min, sqr_max), xlabel = "Year", color = 'purple', 
                        ylabel= "Sale Price per SqrFt")

In [ ]:
# Calculate and plot line chart for average montly rent
avg_rent_df = average_price_group['gross_rent'].reset_index()

rent_std = avg_rent_df.gross_rent.std()
rent_min = min(avg_rent_df.gross_rent) - rent_std
rent_max = max(avg_rent_df.gross_rent) + rent_std

#Optional challenge: Plot each line chart in a different color.

avg_rent_df.hvplot.line(x='year', y='gross_rent',title= "Average Gross Rent by Year", 
                        ylim = (rent_min, rent_max), xlabel = "Year", 
                        color = 'red', ylabel= "Gross Rent")

- - - 

## Average Prices by Neighborhood

In this section, you will use hvplot to create two interactive visulizations of average prices with a dropdown selector for the neighborhood. The first visualization will be a line plot showing the trend of average price per square foot over time for each neighborhood.  The second will be a line plot showing the trend of average montly rent over time for each neighborhood.

**Hint:** It will be easier to create a new DataFrame from grouping the data and calculating the mean prices for each year and neighborhood

In [ ]:
# Group by year and neighborhood and then create a new dataframe of the mean values
# This will make year and neighborhood as indexes, and will perform mean function on the 
# sale_price_sqr_foot, 'housing_units' and gross_rent columns

neighborhood_grp = (sfo_data[['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent']]
                         .groupby(['year', 'neighborhood'])).mean()

# This code will create pandas DataFrame and will reset index for chart 

neighborhood_grp_df = pd.DataFrame(neighborhood_grp).reset_index()

# Rename column name (Neighborhood) for better visualization

neighborhood_grp_df.rename(columns={'neighborhood': 'Neighborhood'}, inplace = True)
neighborhood_grp_df.head(10)

In [ ]:
# Create interactive hvplot  for average sales price per square foot using 
# groupby function to make dropdown selector for neighborhood interact feature
   
neighborhood_grp_df.hvplot.line(x= 'year',
                                y='sale_price_sqr_foot',
                                xlabel= 'Year',
                                ylabel= 'Average Sale Price per SqrFt',
                                groupby= 'Neighborhood'
                                )

In [ ]:
# Create interactive hvplot  for Average Gross Rent using 
# groupby function to make dropdown selector for neighborhood interact feature
# Note: Some of the neighborhoods have the same average gross rent
    
neighborhood_grp_df.hvplot.line(x= 'year',
                                y='gross_rent',
                                xlabel= 'Year',
                                ylabel= 'Average Gross Rent',
                                groupby= 'Neighborhood'
                                )

## The Top 10 Most Expensive Neighborhoods

In this section, you will need to calculate the mean sale price per square foot for each neighborhood and then sort the values to obtain the top 10 most expensive neighborhoods on average. Plot the results as a bar chart.

In [ ]:
# Getting the data from the top 10 expensive neighborhoods to own.
# Group by neighborhood to create a new dataframe of the mean values for sale price per square foot, 
# housing_units, and gross rent. Finally reset the index for plotting purposes

neighborhood_max_df = (sfo_data[['neighborhood', 'sale_price_sqr_foot', 'housing_units','gross_rent']]
                   .groupby(['neighborhood'])).mean()
neighborhood_max_grp_df = pd.DataFrame(neighborhood_max_df).reset_index()

# Round sale price per square foot values (for better visualization) and then sort by descending 

neighborhood_max_grp_df['sale_price_sqr_foot'] = round(neighborhood_max_grp_df['sale_price_sqr_foot'], 2)
neighborhood_max_grp_df = neighborhood_max_grp_df.sort_values(['sale_price_sqr_foot'], ascending = False)
 
# assign neighborhood_max_grp_df top 10 values

neighborhood_max_grp_df_top10 = neighborhood_max_grp_df.head(10)
neighborhood_max_grp_df_top10

In [ ]:
# Plotting data from the top 10 expensive neighborhoods 

neighborhood_max_grp_df_top10.hvplot.bar(
    x = 'neighborhood',
    y = 'sale_price_sqr_foot',
    xlabel = "Neighborhoods",
    ylabel = 'Average Sales price per SqrFt',
    title = 'Top 10 Most Expensive Neighborhoods in San Francisco',
    rot = 90,
    height = 400
)

- - - 

## Comparing cost to purchase versus rental income

In this section, you will use `hvplot` to create an interactive visualization with a dropdown selector for the neighborhood. This visualization will feature a side-by-side comparison of average price per square foot versus average montly rent by year.  

**Hint:** Use the `hvplot` parameter, `groupby`, to create a dropdown selector for the neighborhood.

In [ ]:
# Fetch the previously generated DataFrame that was grouped by year and neighborhood

combined_sale_rental_average = neighborhood_grp_df
combined_sale_rental_average.drop(columns= ['housing_units'], inplace = True)

In [ ]:
combined_sale_rental_average.rename(columns= {
    'year': 'Year', 'gross_rent': 'Gross Rent', 'sale_price_sqr_foot': 'Sale Price per SqrFt'}, inplace = True)
combined_sale_rental_average.set_index('Year', inplace = True)
combined_sale_rental_average.head(10)

In [ ]:
# Plotting the data 

combined_sale_rental_average.hvplot.bar(
    xlabel= 'Neighborhoods',
    ylabel= 'Dollar Amount',
    groupby= 'Neighborhood',
    rot=90,
    height = 400
    )

- - - 

## Neighborhood Map

In this section, you will read in neighborhoods location data and build an interactive map with the average house value per neighborhood. Use a `scatter_mapbox` from Plotly express to create the visualization. Remember, you will need your Mapbox API key for this.

### Load Location Data

In [ ]:
# Load neighborhoods coordinates data

location_path = Path("Data/neighborhoods_coordinates.csv")
sf_location_data = pd.read_csv(location_path)
sf_location_data.head()

### Data Preparation

You will need to join the location data with the mean values per neighborhood.

1. Calculate the mean values for each neighborhood.

2. Join the average values with the neighborhood locations.

In [ ]:
# Calculate the mean values for each neighborhood

sf_combined_mean_df = (sfo_data[['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent']]
                       .groupby(['neighborhood'])).mean().reset_index()
sf_combined_mean_df.head()

In [ ]:
# Join the average values with the neighborhood locations

neighborhood_combined_loca_df = pd.concat([sf_location_data, sf_combined_mean_df], axis = 'columns', join = 'inner')
neighborhood_combined_loca_df.drop(columns=['neighborhood'], inplace = True)
neighborhood_combined_loca_df.head()

### Mapbox Visualization

Plot the average values per neighborhood using a Plotly express `scatter_mapbox` visualization.

In [ ]:
# Set the mapbox access token
px.set_mapbox_access_token(map_box_api)

sf_location_map = px.scatter_mapbox(
    neighborhood_combined_loca_df,
    lat = 'Lat',
    lon = 'Lon',
    size = 'sale_price_sqr_foot',
    hover_name = 'Neighborhood',
    color = 'gross_rent',
    color_continuous_scale=px.colors.cyclical.IceFire,
    mapbox_style='carto-positron',
    size_max=15,
    zoom=11,
    height = 600,
    width = 800,
    title = 'Average Sale Price Per Square Foot and Gross Rent in San Francisco'
)

sf_location_map.show()

- - -

## Cost Analysis - Optional Challenge

In this section, you will use Plotly express to create visualizations that investors can use to interactively filter and explore various factors related to the house value of the San Francisco's neighborhoods. 

### Create a DataFrame showing the most expensive neighborhoods in San Francisco by year
### Create a parallel coordinates plot and parallel categories plot of most expensive neighborhoods in San Francisco per year

In [ ]:
# Used previously created dataframe. Prep data for better visualization purposes

expensive_neighborhoods_per_year = neighborhood_max_grp_df
expensive_neighborhoods_per_year['sale_price_sqr_foot'] = round(expensive_neighborhoods_per_year['sale_price_sqr_foot'], 2)
expensive_neighborhoods_per_year['gross_rent'] = round(expensive_neighborhoods_per_year['gross_rent'], 2)
expensive_neighborhoods_per_year.rename(columns={
    'neighborhood': 'Neighborhood', 'sale_price_sqr_foot': 'Average Sale Price per SqrFt', 
    'housing_units': 'Average Housing Units', 'gross_rent': 'Average Gross Rent'}, inplace = True)
expensive_neighborhoods_per_year = expensive_neighborhoods_per_year.head(10)
expensive_neighborhoods_per_year

In [ ]:
# Parallel Categories Plot

px.parallel_categories(expensive_neighborhoods_per_year, 
                        dimensions=['Neighborhood', 'Average Sale Price per SqrFt', 'Average Housing Units', 'Average Gross Rent'],
                        color='Average Sale Price per SqrFt',
                        title= 'Parallel Categories Analysis of Most Expensive SF Neighborhoods')

In [ ]:
# Parallel Coordinates Plot

px.parallel_coordinates(expensive_neighborhoods_per_year, 
                        dimensions=['Average Sale Price per SqrFt', 'Average Housing Units', 'Average Gross Rent'],
                        color='Average Sale Price per SqrFt',
                        title= 'Parallel Coordinates Analysis of Most Expensive SF Neighborhoods')

### Create a sunburst chart to conduct a costs analysis of most expensive neighborhoods in San Francisco per year

In [ ]:
# Prep data for sunburst chart

c_sunburst = sfo_data[sfo_data["neighborhood"].isin(neighborhood_max_grp_df_top10["neighborhood"])]
c_sunburst.sort_values('sale_price_sqr_foot', ascending = False, inplace = True)
c_sunburst.reset_index(inplace = True)
c_sunburst.rename(columns={
    'year': 'Year', 'neighborhood': 'Neighborhood', 'sale_price_sqr_foot': 'Average Sale Price per SqrFt', 
    'housing_units': 'Average Housing Units', 'gross_rent': 'Average Gross Rent'}, inplace = True)

c_sunburst.head()

In [ ]:
# Plot sunburst chart 

fig = px.sunburst(
    c_sunburst,
    path = ['Year', 'Neighborhood'],
    color = 'Average Gross Rent',
    color_continuous_scale=px.colors.cyclical.IceFire,
    width = 750,
    height = 750,
    title= "Cost Analysis of Most Expensive Neighborhoods in San Francisco"
)
fig.show()